In [1]:
from Demand_restoration import *

## Reading input files

In [2]:
PRODUCT_ATTR = pd.read_csv('data/DPS_PRODUCT_ATTR.csv')
STOCK = pd.read_csv('data/DPS_STOCK.csv')
SELL_IN = pd.read_csv('data/DPS_SELL_IN.csv')
SELL_OUT = pd.read_csv('data/DPS_SELL_OUT.csv')
PROMO = pd.read_csv('data/DPS_PROMO.csv')

PRODUCT = pd.read_csv('data/DPS_PRODUCT.csv')
LOCATION = pd.read_csv('data/DPS_LOCATION.csv')
CUSTOMER = pd.read_csv('data/DPS_CUSTOMER.csv')
DISTR_CHANNEL = pd.read_csv('data/DPS_DISTR_CHANNEL.csv')

hierarchies = {
    'PRODUCT' : PRODUCT,
    'LOCATION' : LOCATION,
    'CUSTOMER' : CUSTOMER,
    'DISTR_CHANNEL' : DISTR_CHANNEL
}

## Config example

In [3]:
DR_PARAMETERS = {
    'DR_OBS_NUM' : 30,
    'DR_LIFECYCLE_MARGIN' : 7,
    'DR_PERIOD_LENGTH' : 30,
    'DEF_INV_TRSHD' : 1000,
    'DEF_QTY_TRSHD' : 10,
    'MIN_SALES_QTY_DAY' : 0,
    'MIN_PROLONG_HIST_MONTH' : 3,
    'MAX_PROLONG_HIST_MONTH' : 24,
    'HIGH_TURNOVER_TRSHD' : 1000,
    'MIN_ND_DAYS' : 1
}

TGT_VAR_CONFIG = {
    'tgt_type' : 'pos',
    'value_src' : 'SALES_QTY',
    'act_flag' : 1,
    'dr_scen' : 3,
    'link_with_stock' : 1,
    'link_with_promo' : 0,
    'link_with_price' : 1,
    'vf_product_lvl' : 7,
    'vf_location_lvl' : 1,
    'vf_customer_lvl' : 5,
    'vf_distr_channel_lvl' : 1,
    'vf_time_lvl' : 'MONTH',
    'ml_product_lvl' : 7,
    'ml_location_lvl' : 5,
    'ml_customer_lvl' : 4,
    'ml_distr_channel_lvl' : 1,
    'ml_time_lvl' : 'WEEK'
}

SEASONAL_FLAG_CONFIG = {
    'EA' : 1,
    'NY' : 1,
    'BA' : 1,
    'XX' : 1,
    'YY' : 1
}

SEASONAL_FLAG_CONFIG = pd.DataFrame([SEASONAL_FLAG_CONFIG]).T.reset_index().rename(columns={'index':'PRODUCT_ATTR_NAME', 0:'PRODUCT_ATTR_VALUE'})

IB_MAX_DT = datetime.datetime(2100, 1, 1)
IB_UPDATE_HISTORY_DEPTH = 30
IB_HIST_START_DT = datetime.datetime(2022, 6, 15)
IB_HIST_END_DT = datetime.datetime(2022, 7, 11)
HIGH_TURNOVER_TRSHD = 100

## Generating data

In [4]:
SALES, FORECAST_FLAG, RESTORED_DEMAND, PROMO, STOCK, PRODUCT_ATTR = generate_data(
    DR_PARAMETERS, TGT_VAR_CONFIG, IB_MAX_DT, IB_UPDATE_HISTORY_DEPTH, IB_HIST_START_DT,
    IB_HIST_END_DT, hierarchies, STOCK, PROMO, PRODUCT_ATTR
)

## Running the algorighm

In [5]:
df = demand_restoration_algorithm(
    DR_PARAMETERS, TGT_VAR_CONFIG, IB_MAX_DT, IB_UPDATE_HISTORY_DEPTH, IB_HIST_START_DT,
    IB_HIST_END_DT, hierarchies, STOCK, PROMO, SALES, FORECAST_FLAG, RESTORED_DEMAND,
    HIGH_TURNOVER_TRSHD, PRODUCT_ATTR, SEASONAL_FLAG_CONFIG
)

In [7]:
df

,PERIOD_DT,PERIOD_END_DT,PRODUCT_ID,LOCATION_ID,CUSTOMER_ID,DISTR_CHANNEL_ID,STATUS,TGT_QTY,PROMO_FLG,PROMO_ID,...,PROMO_PRICE,SUM_TGT_QTY,CLOSED_FLG,DEFICIT_FLG1,mean,std,COUNT_NONDEFECIT_DAYS,Threshold,DEFICIT_FLG2,TGT_QTY_R
6,2022-06-15,2022-06-22,80002,600002,6000002,1,maturity,818.0,0.0,1.0,...,NaN,139087.0,1.0,0.0,818.0,1.0,1.0,816.0,0.0,818.0
7,2022-06-15,2022-06-22,80002,600002,6000003,1,maturity,380.0,1.0,1.0,...,NaN,139087.0,1.0,0.0,380.0,1.0,1.0,378.0,0.0,380.0
8,2022-06-15,2022-06-22,80002,600002,6000004,1,maturity,249.0,0.0,1.0,...,NaN,139087.0,1.0,0.0,249.0,1.0,1.0,247.0,0.0,249.0
9,2022-06-15,2022-06-22,80002,600002,6000015,1,maturity,166.0,0.0,1.0,...,NaN,139087.0,1.0,0.0,166.0,1.0,1.0,164.0,0.0,166.0
10,2022-06-15,2022-06-22,80002,600002,6000018,1,maturity,820.0,0.0,1.0,...,NaN,139087.0,1.0,0.0,820.0,1.0,1.0,818.0,0.0,820.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,2022-06-15,2022-06-22,80072,600006,6000012,1,maturity,794.0,1.0,1.0,...,NaN,29618.0,1.0,0.0,794.0,1.0,1.0,792.0,0.0,794.0
1848,2022-06-15,2022-06-22,80072,600006,6000014,1,maturity,613.0,1.0,1.0,...,NaN,29618.0,1.0,0.0,613.0,1.0,1.0,611.0,0.0,613.0
1849,2022-06-15,2022-06-22,80072,600006,6000015,1,maturity,491.0,0.0,1.0,...,NaN,29618.0,1.0,0.0,491.0,1.0,1.0,489.0,0.0,491.0
1850,2022-06-15,2022-06-22,80072,600006,6000018,1,maturity,734.0,0.0,1.0,...,NaN,29618.0,1.0,0.0,734.0,1.0,1.0,732.0,0.0,734.0
